In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from pymc3 import *
import yfinance as yf
import pandas as pd
import datetime
import tkinter as tk
from tkinter import simpledialog

#Define ticker and date...
ticker = input('Input Ticker: ')

date_from = input('Date from (YYYY-MM-DD): ')
year, month, day = map(int, date_from.split('-'))
date1 = datetime.date(year, month, day)

date_till = input('Date till (YYYY-MM-DD): ')
year, month, day = map(int, date_till.split('-'))
date2 = datetime.date(year, month, day)

#Generate price data
prices_df = pd.DataFrame(yf.download(ticker, fields=['Adj Close'], start=date1, end=date2)['Adj Close'])

#plot?
#plt.plot(prices_df);
#plt.legend([ticker])
#plt.show()

#Calcualting Log-Returns
ticker_ts = prices_df['Adj Close']
ticker_pc = ticker_ts.pct_change()
ticker_log = np.log(ticker_ts) - np.log(ticker_ts.shift(1))
tickerlog_nona = ticker_log.dropna()
Y = tickerlog_nona

#plt.hist(tickerlog_nona, bins = 200);
#plt.show()

#Dialogs for priors
ROOT = tk.Tk()

ROOT.withdraw()
# the input dialog
dist = input('Enter prior distribution! eg. Nromal, StudentT...): ')

if dist == str('Normal'):
    mean = float(input('Enter posterier mu!: '))
    stdev = float(input('Enter posterior sigma!: '))
elif dist == str('StudentT'):
    mean = float(input('Enter posterior mu!: '))
    stdev = float(input('Enter posterior simga!: '))
    v = float(input('Enter posterior nu!: '))
else: 
        print('ERROR: Value not accepted!')
        
#MAKE MODEL
        
model_new = Model()
    
if dist == str('StudentT'):
   
    with model_new:
    
            #Priors for unknown model parameters
            mu = StudentT('mu', nu=v, mu=mean, sigma=stdev)
            sigma = HalfStudentT('sigma', sigma=stdev, nu=v)
            nu = Exponential('nu', 1./10, testval = 5.)

    
            #Likelyhood (sampling distribution of observations)
            Y2_obs = StudentT("Y2_obs", mu = mu, sigma = sigma , nu = nu, observed = Y)
            returns_sim = StudentT('returns_sim', nu=nu, mu=mu, sigma=sigma)
            
elif dist == str('Normal'):
    
    with model_new:
    
            #Priors for unknown model parameters
            mu = Normal("mu", mu=mean, sigma=stdev)
            sigma = HalfNormal('sigma', sigma=stdev)
            returns_sim = Normal('returns_sim', mu=mu, sigma=sigma)
    
    
            #Likelyhood (sampling distribution of observations)
            Y_obs = Normal("Y_obs", mu = mu, sigma = sigma , observed = Y)
            
#Sampling
with model_new:
    # draw 200 posterior samples
    trace = sample(200, return_inferencedata=False)

#Traceplot
with model_new:
    az.plot_trace(trace);

with model_new:
    post_pred = sample_posterior_predictive(trace.posterior)

# add posterior predictive to the InferenceData
az.concat(trace, az.from_pymc3(posterior_predictive=post_pred), inplace=True)

fig, ax = plt.subplots(figsize=(15,8))
az.plot_ppc(trace, ax=ax)
ax.hist(tickerlog_nona, bins=300, alpha=.3, density=True, label=ticker + str('log returns'))
ax.legend(fontsize=10);
plt.xlim([-.2,.2]);
plt.ylim([0,50])
plt.show()

/usr/local/lib/python3.9/site-packages/theano/configdefaults.py:696: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn(
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
